# 📦 Imports & Config

In [1]:
import os, time, json, socket, warnings, collections
warnings.filterwarnings("ignore", category=UserWarning)

import cv2
import numpy as np
import torch
import torch.nn as nn
import torchaudio
import sounddevice as sd
import mediapipe as mp

UDP_IP            = "127.0.0.1"
UDP_PORT          = 5500
SAMPLE_RATE       = 16000
VOICE_N_MFCC      = 40
VAD_RMS_THRESH    = 0.004
CYCLE_SECS        = 5
SMOOTH_WINDOW     = 3
SHOCK_CONF_THRESH = 0.22
DEBUG             = True

ACTIONS  = ["stop", "up", "down", "left", "right", "forward"]
EMOTIONS = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]


# 📡 UDP Socket

In [2]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
print(f"📡 UDP socket ready (IP={UDP_IP}, PORT={UDP_PORT})")


📡 UDP socket ready (IP=127.0.0.1, PORT=5500)


# 📷 Camera Setup

In [3]:
def get_camera():
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    if not cap.isOpened():
        raise RuntimeError("❌ Could not open webcam.")
    print("📹 Camera started successfully.")
    return cap

face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


# 🎙️ Audio / MFCC Setup

In [4]:
_mfcc_transform = torchaudio.transforms.MFCC(
    sample_rate=SAMPLE_RATE,
    n_mfcc=VOICE_N_MFCC,
    melkwargs={"n_fft": 512, "hop_length": 160, "n_mels": VOICE_N_MFCC, "center": False}
)

def record_audio_get_mfcc(duration=2):
    print(f"🎙️ Recording {duration}s...")
    try:
        sd.default.reset()
    except Exception:
        pass
    audio = sd.rec(int(duration * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1, dtype='float32')
    sd.wait()
    rms = float(np.sqrt(np.mean(np.square(audio))))
    if rms < VAD_RMS_THRESH:
        print(f"🤫 Voice inactive (RMS={rms:.5f})")
        return None
    waveform = torch.tensor(audio.T)
    mfcc = _mfcc_transform(waveform).mean(dim=2)
    mfcc = (mfcc - mfcc.mean()) / (mfcc.std() + 1e-6)
    print("✅ [mic] Audio processed.")
    return mfcc


# 🧠 Model Definitions

In [5]:
class GestureNet(nn.Module):
    def __init__(self, num_classes=6):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(63, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, num_classes)
        )
    def forward(self, x): return self.fc(x)

class VoiceNet_CNN(nn.Module):
    def __init__(self, num_classes=6):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 8, 3, padding=1), nn.ReLU(),
            nn.Conv1d(8, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(320, 64), nn.ReLU(),
            nn.Linear(64, num_classes)
        )
    def forward(self, x):
        if len(x.shape) == 2: x = x.unsqueeze(1)
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class EmotionNet(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8,16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16,32,3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(32*6*6, 128), nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


# ⚙️ Load Models

In [6]:
def safe_load(model, path, name):
    if not os.path.exists(path):
        print(f"⚠️ {name} weights not found at {path}. Using randomly initialized model.")
        return
    try:
        state = torch.load(path, map_location="cpu")
        msg = model.load_state_dict(state, strict=False)
        missing = getattr(msg, "missing_keys", [])
        unexpected = getattr(msg, "unexpected_keys", [])
        if missing or unexpected:
            print(f"ℹ️ {name}: Ignored {len(missing)} missing and {len(unexpected)} unexpected keys.")
        print(f"✅ {name} model loaded.")
    except Exception as e:
        print(f"⚠️ Could not load {name} model: {e}")

gesture_model = GestureNet()
voice_model   = VoiceNet_CNN()
emotion_model = EmotionNet()

safe_load(gesture_model, "models/gesture_model.pth", "Gesture")
safe_load(voice_model,   "models/voice_model.pth",   "Voice")
safe_load(emotion_model, "models/emotion_model.pth", "Emotion")

gesture_model.eval(); voice_model.eval(); emotion_model.eval()

gesture_hist = collections.deque(maxlen=SMOOTH_WINDOW)
voice_hist   = collections.deque(maxlen=SMOOTH_WINDOW)


✅ Gesture model loaded.
ℹ️ Voice: Ignored 2 missing and 2 unexpected keys.
✅ Voice model loaded.
ℹ️ Emotion: Ignored 6 missing and 2 unexpected keys.
✅ Emotion model loaded.


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_18472\373159606.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(path, map_location="cpu")


# ✋ Gesture Detection

In [7]:
def get_gesture_cmd(frame):
    with mp.solutions.hands.Hands(
        max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as hands:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = hands.process(rgb)
        if not res.multi_hand_landmarks:
            return "stop", 0.0
        landmarks = np.array([[lm.x, lm.y, lm.z] for lm in res.multi_hand_landmarks[0].landmark])
        wrist = landmarks[0]
        norm = (landmarks - wrist) / (np.linalg.norm(landmarks - wrist) + 1e-6)
        lm_flat = torch.tensor(norm.flatten(), dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            out = gesture_model(lm_flat)
            probs = torch.softmax(out, dim=1).numpy().flatten()
        cmd = ACTIONS[int(np.argmax(probs))]
        conf = float(np.max(probs))
        gesture_hist.append(cmd)
        smooth = max(set(gesture_hist), key=gesture_hist.count)
        if DEBUG: print(f"🖐️ Gesture → {cmd} (conf={conf:.2f}) | Smoothed={smooth}")
        return smooth, conf


# 🎧 Voice Detection

In [8]:
def get_voice_cmd(mfcc):
    if mfcc is None: return None, 0.0
    with torch.no_grad():
        out = voice_model(mfcc)
        probs = torch.softmax(out, dim=1).numpy().flatten()
    cmd = ACTIONS[int(np.argmax(probs))]
    conf = float(np.max(probs))
    voice_hist.append(cmd)
    smooth = max(set(voice_hist), key=voice_hist.count)
    if DEBUG: print(f"🎧 Voice → {cmd} (conf={conf:.2f}) | Smoothed={smooth}")
    return smooth, conf


# 😲 Emotion Detection

In [9]:
def get_emotion_cmd(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.1, 5)
    if len(faces) == 0:
        return "none", 0.0
    x, y, w, h = faces[0]
    pad = int(0.2 * h)
    y1, y2 = max(0, y - pad), min(gray.shape[0], y + h + pad)
    x1, x2 = max(0, x - pad), min(gray.shape[1], x + w + pad)
    face = cv2.resize(gray[y1:y2, x1:x2], (48,48))
    face_t = torch.tensor(face, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0
    with torch.no_grad():
        out = emotion_model(face_t)
        probs = torch.softmax(out, dim=1).numpy().flatten()
    emotion = EMOTIONS[int(np.argmax(probs))]
    conf = float(np.max(probs))
    if emotion == "surprise" and conf >= SHOCK_CONF_THRESH:
        if DEBUG: print(f"⚡ Shock detected! (conf={conf:.2f}) → LAND")
        return "land", conf
    if DEBUG: print(f"🙂 Emotion → {emotion} (conf={conf:.2f})")
    return "none", conf


# 🚀 Main Loop

In [10]:
def run():
    cap = get_camera()
    last_cycle = 0
    try:
        while True:
            ret, frame = cap.read()
            if not ret: break
            overlay = frame.copy()
            cv2.putText(overlay, "Drone Control Active", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            if time.time() - last_cycle > CYCLE_SECS:
                print("\n============================")
                print("🧠 Processing new command cycle...")
                last_cycle = time.time()
                mfcc = record_audio_get_mfcc(2)
                voice_cmd, v_conf = get_voice_cmd(mfcc)
                gesture_cmd, g_conf = get_gesture_cmd(frame)
                emo_cmd, e_conf = get_emotion_cmd(frame)
                if emo_cmd == "land":
                    final, src = "land", "face"
                    print("🛬 Shock face detected — landing and terminating program.")
                    payload = {"source": src, "command": final, "timestamp": time.strftime("%H:%M:%S")}
                    sock.sendto(json.dumps(payload).encode(), (UDP_IP, UDP_PORT))
                    break
                elif gesture_cmd != "stop":
                    final, src = gesture_cmd, "camera"
                elif voice_cmd not in [None, "stop"]:
                    final, src = voice_cmd, "mic"
                else:
                    final, src = "hover", "system"
                print(f"📊 Summary: Gesture={gesture_cmd}, Voice={voice_cmd}, Emotion={emo_cmd}")
                print(f"🎯 Final Decision: [{src.upper()}] → {final.upper()}")
                payload = {"source": src, "command": final, "timestamp": time.strftime("%H:%M:%S")}
                sock.sendto(json.dumps(payload).encode(), (UDP_IP, UDP_PORT))
            cv2.imshow("Drone Feed", overlay)
            if cv2.waitKey(1) & 0xFF == ord('q'): break
    except KeyboardInterrupt:
        print("\n🛑 Interrupted.")
    finally:
        cap.release(); cv2.destroyAllWindows(); sock.close()
        print("✅ Camera + UDP closed.")


# ▶️ Run Program

In [11]:
if __name__ == "__main__":
    run()


📹 Camera started successfully.

🧠 Processing new command cycle...
🎙️ Recording 2s...
✅ [mic] Audio processed.
🎧 Voice → up (conf=0.30) | Smoothed=up
🙂 Emotion → sad (conf=0.16)
📊 Summary: Gesture=stop, Voice=up, Emotion=none
🎯 Final Decision: [MIC] → UP

🧠 Processing new command cycle...
🎙️ Recording 2s...
✅ [mic] Audio processed.
🎧 Voice → up (conf=0.30) | Smoothed=up
📊 Summary: Gesture=stop, Voice=up, Emotion=none
🎯 Final Decision: [MIC] → UP

🧠 Processing new command cycle...
🎙️ Recording 2s...
✅ [mic] Audio processed.
🎧 Voice → up (conf=0.30) | Smoothed=up
📊 Summary: Gesture=stop, Voice=up, Emotion=none
🎯 Final Decision: [MIC] → UP

🧠 Processing new command cycle...
🎙️ Recording 2s...
🤫 Voice inactive (RMS=0.00004)
🖐️ Gesture → up (conf=0.20) | Smoothed=up
📊 Summary: Gesture=up, Voice=None, Emotion=none
🎯 Final Decision: [CAMERA] → UP

🧠 Processing new command cycle...
🎙️ Recording 2s...
🤫 Voice inactive (RMS=0.00057)
📊 Summary: Gesture=stop, Voice=None, Emotion=none
🎯 Final Decisi